In [1]:
import numpy as np
import torch
import torchvision
import cv2
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor

In [3]:
model = torchvision.models.mobilenet_v3_small(pretrained=False, progress=False, )
model.state_dict = torch.load('../model/mobilenet_v3_small.pth')
model.eval()

MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride

In [8]:
batch_size = 1
channels = 3
height = 1024
width = 1024
image = cv2.imread('../web/uploads/photo_2022-04-04_16.16.28.jpeg')
resized_image = cv2.resize(image, (height, width))
transform = ToTensor()

In [20]:
prediction = torch.argmax(model.forward(torch.unsqueeze(transform(resized_image), 0)).data).numpy()

44


In [10]:
import json

In [11]:
with open('classes.json', 'r') as f:
    classes = json.load(f)

In [12]:
classes[torch.argmax(model.forward(torch.unsqueeze(transform(resized_image), 0)).data)]

'alligator lizard'

In [72]:
dummy_input = torch.randn(1, channels, height, width)

input_names = [ "actual_input_1" ]
output_names = [ "output1" ]

torch.onnx.export(model, dummy_input, "mobilenet_v3_small.onnx", verbose=True, input_names=input_names, output_names=output_names)

graph(%actual_input_1 : Float(1, 3, 1024, 1024, strides=[3145728, 1048576, 1024, 1], requires_grad=0, device=cpu),
      %features.1.block.1.fc1.weight : Float(8, 16, 1, 1, strides=[16, 1, 1, 1], requires_grad=1, device=cpu),
      %features.1.block.1.fc1.bias : Float(8, strides=[1], requires_grad=1, device=cpu),
      %features.1.block.1.fc2.weight : Float(16, 8, 1, 1, strides=[8, 1, 1, 1], requires_grad=1, device=cpu),
      %features.1.block.1.fc2.bias : Float(16, strides=[1], requires_grad=1, device=cpu),
      %features.4.block.2.fc1.weight : Float(24, 96, 1, 1, strides=[96, 1, 1, 1], requires_grad=1, device=cpu),
      %features.4.block.2.fc1.bias : Float(24, strides=[1], requires_grad=1, device=cpu),
      %features.4.block.2.fc2.weight : Float(96, 24, 1, 1, strides=[24, 1, 1, 1], requires_grad=1, device=cpu),
      %features.4.block.2.fc2.bias : Float(96, strides=[1], requires_grad=1, device=cpu),
      %features.5.block.2.fc1.weight : Float(64, 240, 1, 1, strides=[240, 1, 1, 1

In [75]:
import onnx

onnx_model = onnx.load("mobilenet_v3_small.onnx")
onnx.checker.check_model(onnx_model)